In [20]:
import pandas as pd

In [21]:
# Cesty k súborom
file_control    = r"C:\Users\marti\Diplomovka\DiplomovkaOpravene.xlsx"
file_stanza     = r"C:\Users\marti\Diplomovka\stanza\outputStanza.xlsx"
file_trankit    = r"C:\Users\marti\Diplomovka\Trankit\Trankit_new_words.xlsx"
file_treetagger = r"C:\Users\marti\Diplomovka\TreeTagger\outputTreeTagger.xlsx"
file_udpipe     = r"C:\Users\marti\Diplomovka\UDPipe\UDPipe.xlsx"
file_morphodita = r"C:\Users\marti\Diplomovka\morphodita\words.xlsx"

In [22]:
# ------------------------------
# 1. Vytvorenie agregovaného DataFrame s tokenmi a UPOS tagmi
# ------------------------------

# Načítanie kontrolného datasetu – predpokladáme, že obsahuje stĺpce "Token" a "Xpos1"
control_df = pd.read_excel(file_control)
n = len(control_df)  # počet riadkov podľa kontrolného datasetu

aggregatedUPOS = pd.DataFrame()
aggregatedUPOS['Control_Token'] = control_df['Token'].iloc[:n].reset_index(drop=True)
aggregatedUPOS['Control_UPOS']  = control_df['Xpos1'].iloc[:n].reset_index(drop=True)

# Funkcia, ktorá načíta daný súbor a vráti stĺpce: Token, UPOS tag a ID (ak existuje)
def load_library(file_path, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=None):
    df = pd.read_excel(file_path)
    if n_rows is not None:
        df = df.iloc[:n_rows].reset_index(drop=True)
    token_series = df[token_col]
    upos_series = df[upos_col]
    id_series = df[id_col] if id_col in df.columns else None
    return token_series.reset_index(drop=True), upos_series.reset_index(drop=True), (id_series.reset_index(drop=True) if id_series is not None else None)

# Stanza (predpokladáme: stĺpce "Token", "Xpos1" a "ID")
stanza_token, stanza_upos, stanza_id = load_library(file_stanza, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=n)
aggregatedUPOS['stanza_Token'] = stanza_token
aggregatedUPOS['stanza_UPOS']  = stanza_upos
aggregatedUPOS['stanza_ID']    = stanza_id

# Trankit (predpokladáme: stĺpce "Token", "XPOS1" a "ID")
trankit_token, trankit_upos, trankit_id = load_library(file_trankit, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=n)
aggregatedUPOS['trankit_Token'] = trankit_token
aggregatedUPOS['trankit_UPOS']  = trankit_upos
aggregatedUPOS['trankit_ID']    = trankit_id

# TreeTagger (predpokladáme: stĺpce "Token", "Xpos1" a "ID")
treetagger_token, treetagger_upos, treetagger_id = load_library(file_treetagger, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=n)
aggregatedUPOS['treetagger_Token'] = treetagger_token
aggregatedUPOS['treetagger_UPOS']  = treetagger_upos
aggregatedUPOS['treetagger_ID']    = treetagger_id

# UDPipe (predpokladáme: stĺpce "Token", "Xpos1" a "ID")
udpipe_token, udpipe_upos, udpipe_id = load_library(file_udpipe, token_col="Token", upos_col="Xpos1", id_col="ID", n_rows=n)
aggregatedUPOS['udpipe_Token'] = udpipe_token
aggregatedUPOS['udpipe_UPOS']  = udpipe_upos
aggregatedUPOS['udpipe_ID']    = udpipe_id

# Morphodita – špeciálne: UPOS získame zo stĺpca "Tag" (iba prvé písmeno) a ID je v stĺpci "Sentence_ID"
df_morphodita = pd.read_excel(file_morphodita)
df_morphodita = df_morphodita.iloc[:n].reset_index(drop=True)
aggregatedUPOS['morphodita_Token'] = df_morphodita['Token']
aggregatedUPOS['morphodita_UPOS']  = df_morphodita['Tag'].astype(str).str[0]
aggregatedUPOS['morphodita_Sentence_ID'] = df_morphodita['Sentence_ID']

# Uloženie agregovaného DataFrame do Excelu
aggregatedUPOS.to_excel(r"C:\Users\marti\Diplomovka\agregovanyUPOS.xlsx", index=False)

# ------------------------------
# 2. Porovnanie UPOS tagov s kontrolou
# ------------------------------

# Pre každý zdroj vytvoríme porovnávací stĺpec, ktorý obsahuje True, ak sa UPOS tag daného zdroja rovná Control_UPOS, inak False.
sources = {
    'stanza': 'stanza_UPOS',
    'trankit': 'trankit_UPOS',
    'treetagger': 'treetagger_UPOS',
    'udpipe': 'udpipe_UPOS',
    'morphodita': 'morphodita_UPOS'
}

for lib, upos_col in sources.items():
    comp_col = f'{lib}_Match'
    aggregatedUPOS[comp_col] = aggregatedUPOS[upos_col] == aggregatedUPOS['Control_UPOS']

# Uloženie výsledného DataFrame (s porovnaním) do Excelu
aggregatedUPOS.to_excel(r"C:\Users\marti\Diplomovka\agregovanyUPOS_with_Match.xlsx", index=False)

# Výpis prvých niekoľkých riadkov na kontrolu
print(aggregatedUPOS.head())

  Control_Token Control_UPOS stanza_Token stanza_UPOS  stanza_ID  \
0            Je            V           Je           V          1   
1            už            T           už           T          2   
2         skoro            T        skoro           D          3   
3       október            S      október           S          4   
4             a            O            a           O          5   

  trankit_Token trankit_UPOS trankit_ID treetagger_Token treetagger_UPOS  ...  \
0            Je            V       None               Je               V  ...   
1            už            T       None               už               T  ...   
2         skoro            T       None            skoro               T  ...   
3       október            S       None          október               S  ...   
4             a            O       None                a               O  ...   

   udpipe_UPOS udpipe_ID morphodita_Token  morphodita_UPOS  \
0            V         1               Je 

In [23]:
from collections import Counter
Counter(aggregatedUPOS['treetagger_Match'])

Counter({False: 850, True: 349})

In [24]:
aggregatedUPOS

,Control_Token,Control_UPOS,stanza_Token,stanza_UPOS,stanza_ID,trankit_Token,trankit_UPOS,trankit_ID,treetagger_Token,treetagger_UPOS,...,udpipe_UPOS,udpipe_ID,morphodita_Token,morphodita_UPOS,morphodita_Sentence_ID,stanza_Match,trankit_Match,treetagger_Match,udpipe_Match,morphodita_Match
0,Je,V,Je,V,1,Je,V,None,Je,V,...,V,1,Je,V,1,True,True,True,True,True
1,už,T,už,T,2,už,T,None,už,T,...,T,2,už,X,1,True,True,True,True,False
2,skoro,T,skoro,D,3,skoro,T,None,skoro,T,...,T,3,skoro,D,1,False,True,True,True,False
3,október,S,október,S,4,október,S,None,október,S,...,S,4,október,N,1,True,True,True,True,False
4,a,O,a,O,5,a,O,None,a,O,...,O,5,a,J,1,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,Slovensku,S,Slovensku,S,5,Slovensku,S,None,Slovensku,S,...,S,5,Slovensku,N,99,True,True,True,True,False
1195,je,V,je,V,6,je,V,None,je,V,...,V,6,je,V,99,True,True,True,True,True
1196,v,E,v,E,7,v,E,None,v,E,...,E,7,v,R,99,True,True,True,True,False
1197,Hurbanove,S,Hurbanove,S,8,Hurbanove,S,None,Hurbanove,A,...,S,8,Hurbanove,N,99,True,True,False,True,False


In [25]:
# Zoznam stĺpcov, ktoré obsahujú výsledky porovnania
match_columns = ['stanza_Match', 'trankit_Match', 'treetagger_Match', 'udpipe_Match', 'morphodita_Match']

total_rows = len(aggregatedUPOS)
success_summary = {}

for col in match_columns:
    # .mean() pre booleovské hodnoty vráti podiel True hodnot (pretože True = 1, False = 0)
    percentage = aggregatedUPOS[col].mean() * 100
    success_summary[col] = percentage

print("Percentuálny podiel úspešnosti pre jednotlivé knižnice:")
for lib, pct in success_summary.items():
    print(f"{lib}: {pct:.2f}%")


Percentuálny podiel úspešnosti pre jednotlivé knižnice:
stanza_Match: 96.83%
trankit_Match: 98.08%
treetagger_Match: 29.11%
udpipe_Match: 97.75%
morphodita_Match: 45.04%
